In [2]:
import os
import zipfile
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty or invalid file skipped: {file_path}")

    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")

    return dataPointsList

###############################################################################
# Data Extraction

# Parameters setup
flakyZip = "Dataset/flaky_files.zip"
nonFlakyZip = "Dataset/nonflaky_files.zip"
extractDir = "extracted"
os.makedirs(extractDir, exist_ok=True)

# Extract the zip files
flakyDir = os.path.join(extractDir, 'flaky')
nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
os.makedirs(flakyDir, exist_ok=True)
os.makedirs(nonFlakyDir, exist_ok=True)

extract_zip(flakyZip, flakyDir)
extract_zip(nonFlakyZip, nonFlakyDir)

dataPointsFlaky = getDataPoints(flakyDir)
dataPointsNonFlaky = getDataPoints(nonFlakyDir)
dataPoints = dataPointsFlaky + dataPointsNonFlaky

print(f"Number of flaky documents: {len(dataPointsFlaky)}")
print(f"Number of non-flaky documents: {len(dataPointsNonFlaky)}")
print(f"Total number of documents: {len(dataPoints)}")

if len(dataPoints) == 0:
    raise ValueError("No documents available for vectorization. Please check the input directories.")

# Create labels: 1 for flaky, 0 for non-flaky
dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))


Number of flaky documents: 45
Number of non-flaky documents: 243
Total number of documents: 288


KNN

In [3]:
import os
import time
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef)
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer
from itertools import product

def KNNwithSMOTEThreshold(dataPoints, dataLabelsList, outDir, n_splits):
    # Define parameter grid
    param_grid = {
        'pca__n_components': [150],             # PCA components
        'knn__n_neighbors': [11],                  # Number of neighbors for KNN
        'knn__weights': [ 'distance'],               # Weighting options
        'knn__metric': [ 'cosine']                 # Distance metrics
    }
    
    param_combinations = list(product(*param_grid.values()))
    param_keys = list(param_grid.keys())
    
    # Prepare to store metrics
    metrics_per_combination = []
    
    # Define thresholds to evaluate
    thresholds = np.linspace(0.1, 0.9, 9)  # Thresholds from 0.1 to 0.9
    
    # Setup cross-validation strategy
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Loop over each hyperparameter combination
    for params in param_combinations:
        param_dict = dict(zip(param_keys, params))
        
        # Initialize lists to store metrics per threshold
        threshold_metrics_list = []
        
        # For each fold in cross-validation
        for fold_idx, (train_index, test_index) in enumerate(skf.split(dataPoints, dataLabelsList)):
            X_train = [dataPoints[i] for i in train_index]
            X_test = [dataPoints[i] for i in test_index]
            y_train = [dataLabelsList[i] for i in train_index]
            y_test = [dataLabelsList[i] for i in test_index]
            
            # Define a pipeline with CountVectorizer, PCA, SMOTE, and KNN with current params
            pipeline = ImbPipeline([
                ('vectorizer', CountVectorizer(stop_words=None)),
                ('pca', PCA(n_components=param_dict['pca__n_components'], random_state=42)),
                ('smote', SMOTE(random_state=42)),
                ('knn', KNeighborsClassifier(
                    n_neighbors=param_dict['knn__n_neighbors'],
                    weights=param_dict['knn__weights'],
                    metric=param_dict['knn__metric']))
            ])
            
            # Train the pipeline
            pipeline.fit(X_train, y_train)
            
            # Predict probabilities
            y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
            
            # Iterate over thresholds
            for threshold in thresholds:
                y_pred_threshold = (y_pred_proba >= threshold).astype(int)
                
                # Calculate metrics
                accuracy = accuracy_score(y_test, y_pred_threshold)
                precision = precision_score(y_test, y_pred_threshold, zero_division=1)
                recall = recall_score(y_test, y_pred_threshold, zero_division=1)
                f1 = f1_score(y_test, y_pred_threshold, zero_division=1)
                mcc = matthews_corrcoef(y_test, y_pred_threshold)
                
                # Store fold metrics
                threshold_metrics_list.append({
                    **param_dict,
                    'threshold': threshold,
                    'fold': fold_idx + 1,
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'mcc': mcc
                })
        
        # Calculate average metrics over folds for each threshold
        for threshold in thresholds:
            # Filter metrics for the current threshold
            threshold_metrics = [tm for tm in threshold_metrics_list if tm['threshold'] == threshold]
            
            avg_accuracy = np.mean([tm['accuracy'] for tm in threshold_metrics])
            avg_precision = np.mean([tm['precision'] for tm in threshold_metrics])
            avg_recall = np.mean([tm['recall'] for tm in threshold_metrics])
            avg_f1 = np.mean([tm['f1'] for tm in threshold_metrics])
            avg_mcc = np.mean([tm['mcc'] for tm in threshold_metrics])
            
            # Store the metrics along with parameters and threshold
            metrics_per_combination.append({
                **param_dict,
                'threshold': threshold,
                'accuracy': avg_accuracy,
                'precision': avg_precision,
                'recall': avg_recall,
                'f1': avg_f1,
                'mcc': avg_mcc
            })
    
    # Now, find the parameter combination with the best F1 score
    best_result = max(metrics_per_combination, key=lambda x: x['f1'])
    
    # Save the results to CSV
    df_metrics = pd.DataFrame(metrics_per_combination)
    outFile_metrics = os.path.join(outDir, f"knn-smote-threshold-results.csv")
    df_metrics.to_csv(outFile_metrics, index=False)
    
    print(f"Results saved to: {outFile_metrics}")
    
    # Extract the best parameters, threshold, and metrics
    best_params = {key: best_result[key] for key in param_keys}
    best_threshold = best_result['threshold']
    best_f1 = best_result['f1']
    final_mcc = best_result['mcc']
    
    print("\nBest Parameters, Threshold, and Metrics:")
    print(f"Best Parameters: {best_params}")
    print(f"Best Threshold: {best_threshold}")
    print(f"Best F1 Score: {best_f1}")
    print(f"Final MCC: {final_mcc}")
    
    return best_params, best_threshold, best_f1, final_mcc

# Main Execution for 5-Fold Cross-Validation
outDir = "results"
os.makedirs(outDir, exist_ok=True)

# Run KNN with SMOTE, PCA, and Threshold adjustment using 5-fold cross-validation
print("\nStarting KNN analysis with SMOTE, PCA, and Threshold adjustment for 5-fold cross-validation...")
best_params_5folds, best_threshold_5folds, best_f1_5folds, final_mcc_5folds = KNNwithSMOTEThreshold(
    dataPoints, dataLabelsList, outDir, n_splits=5)

# Display results
print("\nBest results for KNN with SMOTE, PCA, and Threshold adjustment 5-fold cross-validation:")
print(f"Best Parameters: {best_params_5folds}")
print(f"Best Threshold: {best_threshold_5folds}")
print(f"Best F1 Score: {best_f1_5folds}")
print(f"Final MCC: {final_mcc_5folds}")




Starting KNN analysis with SMOTE, PCA, and Threshold adjustment for 5-fold cross-validation...
Results saved to: results\knn-smote-threshold-results.csv

Best Parameters, Threshold, and Metrics:
Best Parameters: {'pca__n_components': 150, 'knn__n_neighbors': 11, 'knn__weights': 'distance', 'knn__metric': 'cosine'}
Best Threshold: 0.7000000000000001
Best F1 Score: 0.6187409700722395
Final MCC: 0.5862234088981968

Best results for KNN with SMOTE, PCA, and Threshold adjustment 5-fold cross-validation:
Best Parameters: {'pca__n_components': 150, 'knn__n_neighbors': 11, 'knn__weights': 'distance', 'knn__metric': 'cosine'}
Best Threshold: 0.7000000000000001
Best F1 Score: 0.6187409700722395
Final MCC: 0.5862234088981968


SVM

In [4]:
import os
import time
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import (precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef)
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer
from itertools import product

def SVM_SMOTE_Threshold(dataPoints, dataLabelsList, outDir, n_splits):
    v0 = time.perf_counter()
    
    # Define parameter grid
    param_grid = {
        'pca__n_components': [220],          # PCA components
        'svm__C': [0.01],            # Regularization parameter
        'svm__kernel': ['linear'] # Kernel types
    }
    
    param_combinations = list(product(*param_grid.values()))
    param_keys = list(param_grid.keys())
    
    # Prepare to store metrics
    metrics_per_combination = []
    
    # Define thresholds to evaluate
    thresholds = np.linspace(0.1, 0.9, 9)  # Thresholds from 0.1 to 0.9
    
    # Setup cross-validation strategy
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Loop over each hyperparameter combination
    for params in param_combinations:
        param_dict = dict(zip(param_keys, params))
        
        # Initialize lists to store metrics per threshold
        threshold_metrics_list = []
        
        # For each fold in cross-validation
        for fold_idx, (train_index, test_index) in enumerate(skf.split(dataPoints, dataLabelsList)):
            X_train = [dataPoints[i] for i in train_index]
            X_test = [dataPoints[i] for i in test_index]
            y_train = [dataLabelsList[i] for i in train_index]
            y_test = [dataLabelsList[i] for i in test_index]
            
            # Define a pipeline with CountVectorizer, PCA, SMOTE, and SVM with current params
            pipeline = ImbPipeline([
                ('vectorizer', CountVectorizer(stop_words=None)),
                ('pca', PCA(n_components=param_dict['pca__n_components'], random_state=42)),
                ('smote', SMOTE(random_state=42)),
                ('svm', SVC(
                    C=param_dict['svm__C'],
                    kernel=param_dict['svm__kernel'],
                    probability=True,
                    random_state=42))
            ])
            
            # Train the pipeline
            pipeline.fit(X_train, y_train)
            
            # Predict probabilities
            y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
            
            # Iterate over thresholds
            for threshold in thresholds:
                y_pred_threshold = (y_pred_proba >= threshold).astype(int)
                
                # Calculate metrics
                accuracy = accuracy_score(y_test, y_pred_threshold)
                precision = precision_score(y_test, y_pred_threshold, zero_division=1)
                recall = recall_score(y_test, y_pred_threshold, zero_division=1)
                f1 = f1_score(y_test, y_pred_threshold, zero_division=1)
                mcc = matthews_corrcoef(y_test, y_pred_threshold)
                
                # Store fold metrics
                threshold_metrics_list.append({
                    **param_dict,
                    'threshold': threshold,
                    'fold': fold_idx + 1,
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'mcc': mcc
                })
        
        # Calculate average metrics over folds for each threshold
        for threshold in thresholds:
            # Filter metrics for the current threshold
            threshold_metrics = [tm for tm in threshold_metrics_list if tm['threshold'] == threshold]
            
            avg_accuracy = np.mean([tm['accuracy'] for tm in threshold_metrics])
            avg_precision = np.mean([tm['precision'] for tm in threshold_metrics])
            avg_recall = np.mean([tm['recall'] for tm in threshold_metrics])
            avg_f1 = np.mean([tm['f1'] for tm in threshold_metrics])
            avg_mcc = np.mean([tm['mcc'] for tm in threshold_metrics])
            
            # Store the metrics along with parameters and threshold
            metrics_per_combination.append({
                **param_dict,
                'threshold': threshold,
                'accuracy': avg_accuracy,
                'precision': avg_precision,
                'recall': avg_recall,
                'f1': avg_f1,
                'mcc': avg_mcc
            })
    
    # Now, find the parameter combination with the best F1 score
    best_result = max(metrics_per_combination, key=lambda x: x['f1'])
    
    # Save the results to CSV
    df_metrics = pd.DataFrame(metrics_per_combination)
    outFile_metrics = os.path.join(outDir, f"svm-smote-threshold-results.csv")
    df_metrics.to_csv(outFile_metrics, index=False)
    
    print(f"Results saved to: {outFile_metrics}")
    
    # Extract the best parameters, threshold, and metrics
    best_params = {key: best_result[key] for key in param_keys}
    best_threshold = best_result['threshold']
    best_f1 = best_result['f1']
    final_mcc = best_result['mcc']
    
    print("\nBest Parameters, Threshold, and Metrics:")
    print(f"Best Parameters: {best_params}")
    print(f"Best Threshold: {best_threshold}")
    print(f"Best F1 Score: {best_f1}")
    print(f"Final MCC: {final_mcc}")
    
    return best_params, best_threshold, best_f1, final_mcc

# Main Execution
outDir = "results"
os.makedirs(outDir, exist_ok=True)

# Run SVM with SMOTE, PCA, and Threshold adjustment using 5-fold cross-validation
print("\nStarting SVM analysis with SMOTE, PCA, and Threshold adjustment for 5-fold cross-validation...")
best_params_5folds, best_threshold_5folds, best_f1_5folds, final_mcc_5folds = SVM_SMOTE_Threshold(
    dataPoints, dataLabelsList, outDir, n_splits=5)

# Display results
print("\nBest results for SVM with SMOTE, PCA, and Threshold adjustment 5-fold cross-validation:")
print(f"Best Parameters: {best_params_5folds}")
print(f"Best Threshold: {best_threshold_5folds}")
print(f"Best F1 Score: {best_f1_5folds}")
print(f"Final MCC: {final_mcc_5folds}")



Starting SVM analysis with SMOTE, PCA, and Threshold adjustment for 5-fold cross-validation...
Results saved to: results\svm-smote-threshold-results.csv

Best Parameters, Threshold, and Metrics:
Best Parameters: {'pca__n_components': 220, 'svm__C': 0.01, 'svm__kernel': 'linear'}
Best Threshold: 0.1
Best F1 Score: 0.7923042901371076
Final MCC: 0.7675125755123265

Best results for SVM with SMOTE, PCA, and Threshold adjustment 5-fold cross-validation:
Best Parameters: {'pca__n_components': 220, 'svm__C': 0.01, 'svm__kernel': 'linear'}
Best Threshold: 0.1
Best F1 Score: 0.7923042901371076
Final MCC: 0.7675125755123265


XGB

In [5]:
import os
import time
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import (precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef)
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer
from itertools import product

def runXGBWithSMOTE_PCA(dataPoints, dataLabelsList, outDir, n_splits):
    v0 = time.perf_counter()
    
    # Define parameter grid
    param_grid = {
        'pca__n_components': [150],     # PCA components
        'xgb__n_estimators': [100],          # Number of boosting rounds
        'xgb__max_depth': [3],               # Maximum depth of a tree
        'xgb__learning_rate': [ 0.5],   # Learning rate
    }
    
    param_combinations = list(product(*param_grid.values()))
    param_keys = list(param_grid.keys())
    
    # Prepare to store metrics
    metrics_per_combination = []
    
    # Define thresholds to evaluate
    thresholds = np.linspace(0.1, 0.9, 9)  # Thresholds from 0.1 to 0.9
    
    # Setup cross-validation strategy
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Loop over each hyperparameter combination
    for params in param_combinations:
        param_dict = dict(zip(param_keys, params))
        
        # Initialize lists to store metrics per threshold
        threshold_metrics_list = []
        
        # For each fold in cross-validation
        for fold_idx, (train_index, test_index) in enumerate(skf.split(dataPoints, dataLabelsList)):
            X_train = [dataPoints[i] for i in train_index]
            X_test = [dataPoints[i] for i in test_index]
            y_train = [dataLabelsList[i] for i in train_index]
            y_test = [dataLabelsList[i] for i in test_index]
            
            # Define a pipeline with CountVectorizer, PCA, SMOTE, and XGBClassifier with current params
            pipeline = ImbPipeline([
                ('vectorizer', CountVectorizer(stop_words=None)),
                ('pca', PCA(n_components=param_dict['pca__n_components'], random_state=42)),
                ('smote', SMOTE(random_state=42)),
                ('xgb', XGBClassifier(
                    n_estimators=param_dict['xgb__n_estimators'],
                    max_depth=param_dict['xgb__max_depth'],
                    learning_rate=param_dict['xgb__learning_rate'],
                    eval_metric='logloss',
                    use_label_encoder=False,
                    random_state=42))
            ])
            
            # Train the pipeline
            pipeline.fit(X_train, y_train)
            
            # Predict probabilities
            y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
            
            # Iterate over thresholds
            for threshold in thresholds:
                y_pred_threshold = (y_pred_proba >= threshold).astype(int)
                
                # Calculate metrics
                accuracy = accuracy_score(y_test, y_pred_threshold)
                precision = precision_score(y_test, y_pred_threshold, zero_division=1)
                recall = recall_score(y_test, y_pred_threshold, zero_division=1)
                f1 = f1_score(y_test, y_pred_threshold, zero_division=1)
                mcc = matthews_corrcoef(y_test, y_pred_threshold)
                
                # Store fold metrics
                threshold_metrics_list.append({
                    **param_dict,
                    'threshold': threshold,
                    'fold': fold_idx + 1,
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'mcc': mcc
                })
        
        # Calculate average metrics over folds for each threshold
        for threshold in thresholds:
            # Filter metrics for the current threshold
            threshold_metrics = [tm for tm in threshold_metrics_list if tm['threshold'] == threshold]
            
            avg_accuracy = np.mean([tm['accuracy'] for tm in threshold_metrics])
            avg_precision = np.mean([tm['precision'] for tm in threshold_metrics])
            avg_recall = np.mean([tm['recall'] for tm in threshold_metrics])
            avg_f1 = np.mean([tm['f1'] for tm in threshold_metrics])
            avg_mcc = np.mean([tm['mcc'] for tm in threshold_metrics])
            
            # Store the metrics along with parameters and threshold
            metrics_per_combination.append({
                **param_dict,
                'threshold': threshold,
                'accuracy': avg_accuracy,
                'precision': avg_precision,
                'recall': avg_recall,
                'f1': avg_f1,
                'mcc': avg_mcc
            })
    
    # Now, find the parameter combination with the best F1 score
    best_result = max(metrics_per_combination, key=lambda x: x['f1'])
    
    # Save the results to CSV
    df_metrics = pd.DataFrame(metrics_per_combination)
    outFile_metrics = os.path.join(outDir, f"xgb-smote-pca-threshold-results.csv")
    df_metrics.to_csv(outFile_metrics, index=False)
    
    print(f"Results saved to: {outFile_metrics}")
    
    # Extract the best parameters, threshold, and metrics
    best_params = {key: best_result[key] for key in param_keys}
    best_threshold = best_result['threshold']
    best_f1 = best_result['f1']
    final_mcc = best_result['mcc']
    
    print("\nBest Parameters, Threshold, and Metrics:")
    print(f"Best Parameters: {best_params}")
    print(f"Best Threshold: {best_threshold}")
    print(f"Best F1 Score: {best_f1}")
    print(f"Final MCC: {final_mcc}")
    
    return best_params, best_threshold, best_f1, final_mcc

# Main Execution for 5-Fold Cross-Validation
outDir = "result"
os.makedirs(outDir, exist_ok=True)

# Run XGBoost with SMOTE, PCA, and Threshold adjustment using 5-fold cross-validation
print("\nStarting XGBoost analysis with SMOTE, PCA, and Threshold adjustment for 5-fold cross-validation...")
best_params_5folds, best_threshold_5folds, best_f1_5folds, final_mcc_5folds = runXGBWithSMOTE_PCA(
    dataPoints, dataLabelsList, outDir, n_splits=5)

# Display results
print("\nBest results for XGBoost with SMOTE, PCA, and Threshold adjustment 5-fold cross-validation:")
print(f"Best Parameters: {best_params_5folds}")
print(f"Best Threshold: {best_threshold_5folds}")
print(f"Best F1 Score: {best_f1_5folds}")
print(f"Final MCC: {final_mcc_5folds}")



Starting XGBoost analysis with SMOTE, PCA, and Threshold adjustment for 5-fold cross-validation...


c:\Users\haha9\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:59:41] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\haha9\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:59:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\haha9\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:59:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are

Results saved to: result\xgb-smote-pca-threshold-results.csv

Best Parameters, Threshold, and Metrics:
Best Parameters: {'pca__n_components': 150, 'xgb__n_estimators': 100, 'xgb__max_depth': 3, 'xgb__learning_rate': 0.5}
Best Threshold: 0.2
Best F1 Score: 0.6614951611855637
Final MCC: 0.6030750704417529

Best results for XGBoost with SMOTE, PCA, and Threshold adjustment 5-fold cross-validation:
Best Parameters: {'pca__n_components': 150, 'xgb__n_estimators': 100, 'xgb__max_depth': 3, 'xgb__learning_rate': 0.5}
Best Threshold: 0.2
Best F1 Score: 0.6614951611855637
Final MCC: 0.6030750704417529


c:\Users\haha9\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:59:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Random Forest

In [10]:
from sklearn.ensemble import RandomForestClassifier

def runRFWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits):
    v0 = time.perf_counter()
    
    # Define parameter grid
    param_grid = {
        'rf__n_estimators': [100],          # Number of trees in the forest
        'rf__max_depth': [20],               # Maximum depth of the tree
        'rf__min_samples_split': [5],            # Minimum number of samples required to split a node
        'rf__min_samples_leaf': [2],              # Minimum number of samples required at a leaf node
        'rf__criterion': ['entropy']         # Function to measure the quality of a split
    }
    
    param_combinations = list(product(*param_grid.values()))
    param_keys = list(param_grid.keys())
    
    # Prepare to store metrics
    metrics_per_combination = []
    
    # Define thresholds to evaluate
    thresholds = np.linspace(0.1, 0.9, 9)  # Thresholds from 0.1 to 0.9
    
    # Setup cross-validation strategy
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Loop over each hyperparameter combination
    for params in param_combinations:
        param_dict = dict(zip(param_keys, params))
        
        # Initialize lists to store metrics per threshold
        threshold_metrics_list = []
        
        # For each fold in cross-validation
        for fold_idx, (train_index, test_index) in enumerate(skf.split(dataPoints, dataLabelsList)):
            X_train = [dataPoints[i] for i in train_index]
            X_test = [dataPoints[i] for i in test_index]
            y_train = [dataLabelsList[i] for i in train_index]
            y_test = [dataLabelsList[i] for i in test_index]
            
            # Define a pipeline with SMOTE and Random Forest with current params
            pipeline = ImbPipeline([
                ('vectorizer', CountVectorizer(stop_words=None)),
                ('smote', SMOTE(random_state=42)),
                ('rf', RandomForestClassifier(
                    n_estimators=param_dict['rf__n_estimators'],
                    max_depth=param_dict['rf__max_depth'],
                    min_samples_split=param_dict['rf__min_samples_split'],
                    min_samples_leaf=param_dict['rf__min_samples_leaf'],
                    criterion=param_dict['rf__criterion'],
                    random_state=42))
            ])
            
            # Train the pipeline
            pipeline.fit(X_train, y_train)
            
            # Predict probabilities
            y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
            
            # Iterate over thresholds
            for threshold in thresholds:
                y_pred_threshold = (y_pred_proba >= threshold).astype(int)
                
                # Calculate metrics
                accuracy = accuracy_score(y_test, y_pred_threshold)
                precision = precision_score(y_test, y_pred_threshold, zero_division=1)
                recall = recall_score(y_test, y_pred_threshold, zero_division=1)
                f1 = f1_score(y_test, y_pred_threshold, zero_division=1)
                mcc = matthews_corrcoef(y_test, y_pred_threshold)
                
                # Store fold metrics
                threshold_metrics_list.append({
                    **param_dict,
                    'threshold': threshold,
                    'fold': fold_idx + 1,
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'mcc': mcc
                })
        
        # Calculate average metrics over folds for each threshold
        for threshold in thresholds:
            # Filter metrics for the current threshold
            threshold_metrics = [tm for tm in threshold_metrics_list if tm['threshold'] == threshold]
            
            avg_accuracy = np.mean([tm['accuracy'] for tm in threshold_metrics])
            avg_precision = np.mean([tm['precision'] for tm in threshold_metrics])
            avg_recall = np.mean([tm['recall'] for tm in threshold_metrics])
            avg_f1 = np.mean([tm['f1'] for tm in threshold_metrics])
            avg_mcc = np.mean([tm['mcc'] for tm in threshold_metrics])
            
            # Store the metrics along with parameters and threshold
            metrics_per_combination.append({
                **param_dict,
                'threshold': threshold,
                'accuracy': avg_accuracy,
                'precision': avg_precision,
                'recall': avg_recall,
                'f1': avg_f1,
                'mcc': avg_mcc
            })
    
    # Now, find the parameter combination with the best F1 score
    best_result = max(metrics_per_combination, key=lambda x: x['f1'])
    
    # Save the results to CSV
    df_metrics = pd.DataFrame(metrics_per_combination)
    outFile_metrics = os.path.join(outDir, f"rf-smote-threshold-results.csv")
    df_metrics.to_csv(outFile_metrics, index=False)
    
    print(f"Results saved to: {outFile_metrics}")
    
    # Extract the best parameters, threshold, and metrics
    best_params = {key: best_result[key] for key in param_keys}
    best_threshold = best_result['threshold']
    best_f1 = best_result['f1']
    final_mcc = best_result['mcc']
    
    print("\nBest Parameters, Threshold, and Metrics:")
    print(f"Best Parameters: {best_params}")
    print(f"Best Threshold: {best_threshold}")
    print(f"Best F1 Score: {best_f1}")
    print(f"Final MCC: {final_mcc}")
    
    return best_params, best_threshold, best_f1, final_mcc

# Main Execution for 5-Fold Cross-Validation
outDir = "results"
os.makedirs(outDir, exist_ok=True)

# Run Random Forest with SMOTE and Threshold adjustment using 5-fold cross-validation
print("\nStarting Random Forest analysis with SMOTE and Threshold adjustment for 5-fold cross-validation...")
best_params_5folds, best_threshold_5folds, best_f1_5folds, final_mcc_5folds = runRFWithSMOTE(
    dataPoints, dataLabelsList, outDir, n_splits=5)

# Display results
print("\nBest results for Random Forest with SMOTE and Threshold adjustment 5-fold cross-validation:")
print(f"Best Parameters: {best_params_5folds}")
print(f"Best Threshold: {best_threshold_5folds}")
print(f"Best F1 Score: {best_f1_5folds}")
print(f"Final MCC: {final_mcc_5folds}")



Starting Random Forest analysis with SMOTE and Threshold adjustment for 5-fold cross-validation...
Results saved to: results\rf-smote-threshold-results.csv

Best Parameters, Threshold, and Metrics:
Best Parameters: {'rf__n_estimators': 100, 'rf__max_depth': 20, 'rf__min_samples_split': 5, 'rf__min_samples_leaf': 2, 'rf__criterion': 'entropy'}
Best Threshold: 0.5
Best F1 Score: 0.822015823873409
Final MCC: 0.7898138504372108

Best results for Random Forest with SMOTE and Threshold adjustment 5-fold cross-validation:
Best Parameters: {'rf__n_estimators': 100, 'rf__max_depth': 20, 'rf__min_samples_split': 5, 'rf__min_samples_leaf': 2, 'rf__criterion': 'entropy'}
Best Threshold: 0.5
Best F1 Score: 0.822015823873409
Final MCC: 0.7898138504372108


Decision Tree

In [11]:
from sklearn.tree import DecisionTreeClassifier

def runDTWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits):
    v0 = time.perf_counter()
    
    # Define parameter grid
    param_grid = {
        'dt__max_depth': [10],
        'dt__min_samples_split': [5],
        'dt__min_samples_leaf': [2],
        'dt__criterion': ['gini'],
        'dt__max_features': [None]
    }
    
    param_combinations = list(product(*param_grid.values()))
    param_keys = list(param_grid.keys())
    
    # Prepare to store metrics
    metrics_per_combination = []
    
    # Define thresholds to evaluate
    thresholds = np.linspace(0.1, 0.9, 9)  # Thresholds from 0.1 to 0.9
    
    # Setup cross-validation strategy
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Loop over each hyperparameter combination
    for params in param_combinations:
        param_dict = dict(zip(param_keys, params))
        
        # Initialize lists to store metrics per threshold
        threshold_metrics_list = []
        
        # For each fold in cross-validation
        for fold_idx, (train_index, test_index) in enumerate(skf.split(dataPoints, dataLabelsList)):
            X_train = [dataPoints[i] for i in train_index]
            X_test = [dataPoints[i] for i in test_index]
            y_train = [dataLabelsList[i] for i in train_index]
            y_test = [dataLabelsList[i] for i in test_index]
            
            # Define a pipeline with SMOTE and Decision Tree with current params
            pipeline = ImbPipeline([
                ('vectorizer', CountVectorizer(stop_words=None)),
                ('smote', SMOTE(random_state=42)),
                ('dt', DecisionTreeClassifier(
                    max_depth=param_dict['dt__max_depth'],
                    min_samples_split=param_dict['dt__min_samples_split'],
                    min_samples_leaf=param_dict['dt__min_samples_leaf'],
                    criterion=param_dict['dt__criterion'],
                    max_features=param_dict['dt__max_features'],
                    random_state=42))
            ])
            
            # Train the pipeline
            pipeline.fit(X_train, y_train)
            
            # Predict probabilities
            y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
            
            # Iterate over thresholds
            for threshold in thresholds:
                y_pred_threshold = (y_pred_proba >= threshold).astype(int)
                
                # Calculate metrics
                accuracy = accuracy_score(y_test, y_pred_threshold)
                precision = precision_score(y_test, y_pred_threshold, zero_division=1)
                recall = recall_score(y_test, y_pred_threshold, zero_division=1)
                f1 = f1_score(y_test, y_pred_threshold, zero_division=1)
                mcc = matthews_corrcoef(y_test, y_pred_threshold)
                
                # Store fold metrics
                threshold_metrics_list.append({
                    **param_dict,
                    'threshold': threshold,
                    'fold': fold_idx + 1,
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'mcc': mcc
                })
        
        # Calculate average metrics over folds for each threshold
        for threshold in thresholds:
            # Filter metrics for the current threshold
            threshold_metrics = [tm for tm in threshold_metrics_list if tm['threshold'] == threshold]
            
            avg_accuracy = np.mean([tm['accuracy'] for tm in threshold_metrics])
            avg_precision = np.mean([tm['precision'] for tm in threshold_metrics])
            avg_recall = np.mean([tm['recall'] for tm in threshold_metrics])
            avg_f1 = np.mean([tm['f1'] for tm in threshold_metrics])
            avg_mcc = np.mean([tm['mcc'] for tm in threshold_metrics])
            
            # Store the metrics along with parameters and threshold
            metrics_per_combination.append({
                **param_dict,
                'threshold': threshold,
                'accuracy': avg_accuracy,
                'precision': avg_precision,
                'recall': avg_recall,
                'f1': avg_f1,
                'mcc': avg_mcc
            })
    
    # Now, find the parameter combination with the best F1 score
    best_result = max(metrics_per_combination, key=lambda x: x['f1'])
    
    # Save the results to CSV
    df_metrics = pd.DataFrame(metrics_per_combination)
    outFile_metrics = os.path.join(outDir, f"dt-smote-threshold-results-{n_splits}-folds.csv")
    df_metrics.to_csv(outFile_metrics, index=False)
    
    print(f"Results saved to: {outFile_metrics}")
    
    # Extract the best parameters, threshold, and metrics
    best_params = {key: best_result[key] for key in param_keys}
    best_threshold = best_result['threshold']
    best_f1 = best_result['f1']
    final_mcc = best_result['mcc']
    
    print("\nBest Parameters, Threshold, and Metrics:")
    print(f"Best Parameters: {best_params}")
    print(f"Best Threshold: {best_threshold}")
    print(f"Best F1 Score: {best_f1}")
    print(f"Final MCC: {final_mcc}")
    
    return best_params, best_threshold, best_f1, final_mcc
# Main Execution for 5-Fold Cross-Validation
outDir = "smote-results-DT-new"
os.makedirs(outDir, exist_ok=True)

# Run Decision Tree with SMOTE and Threshold adjustment using 5-fold cross-validation
print("\nStarting Decision Tree analysis with SMOTE and Threshold adjustment for 5-fold cross-validation...")
best_params_5folds, best_threshold_5folds, best_f1_5folds, final_mcc_5folds = runDTWithSMOTE(
    dataPoints, dataLabelsList, outDir, n_splits=5)

# Display results
print("\nBest results for Decision Tree with SMOTE and Threshold adjustment 5-fold cross-validation:")
print(f"Best Parameters: {best_params_5folds}")
print(f"Best Threshold: {best_threshold_5folds}")
print(f"Best F1 Score: {best_f1_5folds}")
print(f"Final MCC: {final_mcc_5folds}")



Starting Decision Tree analysis with SMOTE and Threshold adjustment for 5-fold cross-validation...
Results saved to: smote-results-DT-new\dt-smote-threshold-results-5-folds.csv

Best Parameters, Threshold, and Metrics:
Best Parameters: {'dt__max_depth': 10, 'dt__min_samples_split': 5, 'dt__min_samples_leaf': 2, 'dt__criterion': 'gini', 'dt__max_features': None}
Best Threshold: 0.1
Best F1 Score: 0.8760130718954249
Final MCC: 0.8639675781078721

Best results for Decision Tree with SMOTE and Threshold adjustment 5-fold cross-validation:
Best Parameters: {'dt__max_depth': 10, 'dt__min_samples_split': 5, 'dt__min_samples_leaf': 2, 'dt__criterion': 'gini', 'dt__max_features': None}
Best Threshold: 0.1
Best F1 Score: 0.8760130718954249
Final MCC: 0.8639675781078721
